# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
dummify = df.select_dtypes(exclude='number').drop(columns='loan_status')
df = pd.get_dummies(df, columns=list(dummify.columns), drop_first=True)
# Preview the data
df.head()


,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,0,0,0,0,1,1,1,0


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
# YOUR CODE HERE
data_scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
data_scaler.fit(X_train)

StandardScaler()

In [11]:
# Scale the training and testing data
# YOUR CODE HERE
X_train = data_scaler.transform(X_train)
X_test = data_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
clf = BalancedRandomForestClassifier(n_estimators = 100,random_state=1)
clf = clf.fit(X_train, y_train)
pred_y = clf.predict(X_test)

In [20]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, pred_y)

0.7558330898821697

In [21]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, pred_y)

array([[   55,    35],
       [ 1702, 15413]])

In [22]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, pred_y))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.61      0.90      0.06      0.74      0.53        90
   low_risk       1.00      0.90      0.61      0.95      0.74      0.57     17115

avg / total       0.99      0.90      0.61      0.94      0.74      0.57     17205



In [23]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = [df.columns[i] for i in indices]
features

['total_pymnt_inv',
 'total_rec_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'collection_recovery_fee',
 'int_rate',
 'verification_status_Verified',
 'installment',
 'mo_sin_old_il_acct',
 'total_acc',
 'annual_inc',
 'out_prncp',
 'mths_since_recent_bc',
 'tax_liens',
 'mort_acc',
 'avg_cur_bal',
 'bc_open_to_buy',
 'tot_coll_amt',
 'open_rv_24m',
 'loan_status',
 'total_bal_il',
 'delinq_amnt',
 'total_bal_ex_mort',
 'open_il_24m',
 'max_bal_bc',
 'pub_rec',
 'issue_d_Jan-2019',
 'all_util',
 'acc_open_past_24mths',
 'num_op_rev_tl',
 'loan_amnt',
 'tot_hi_cred_lim',
 'mo_sin_old_rev_tl_op',
 'num_actv_bc_tl',
 'total_bc_limit',
 'inq_last_6mths',
 'inq_last_12m',
 'revol_bal',
 'delinq_2yrs',
 'num_rev_tl_bal_gt_0',
 'mths_since_rcnt_il',
 'num_bc_tl',
 'num_tl_op_past_12m',
 'num_il_tl',
 'mo_sin_rcnt_rev_tl_op',
 'total_cu_tl',
 'initial_list_status_w',
 'open_il_12m',
 'total_rev_hi_lim',
 'num_rev_accts',
 'num_bc_sats',
 'pct_tl_nvr_dlq',
 'open_acc_6m',
 'mo_sin_rcnt_tl',
 'inq

### Easy Ensemble Classifier

In [24]:
# Train the Classifier
# YOUR CODE HERE
eec = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
eec = eec.fit(X_train, y_train)
pred_y = eec.predict(X_test)

In [25]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test, pred_y)

0.8887038660044795

In [26]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, pred_y)

array([[   74,    16],
       [  767, 16348]])

In [27]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, pred_y))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.82      0.96      0.16      0.89      0.77        90
   low_risk       1.00      0.96      0.82      0.98      0.89      0.80     17115

avg / total       0.99      0.95      0.82      0.97      0.89      0.80     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier

2. Which model had the best recall score?

    Easy Ensemble Classifier

3. Which model had the best geometric mean score?

    Easy Ensemble Classifier

4. What are the top three features?

    'total_pymnt_inv', 'total_rec_prncp', 'out_prncp_inv'